# 🌌 Master Notebook – Pipeline Spectroscopie DR5

## Objectif du pipeline

- Ce notebook télécharge, prépare et journalise des spectres .fits.gz de LAMOST DR5 pour entraîner un modèle de classification.
- Le DatasetBuilder garantit qu'aucun spectre ne sera jamais réutilisé en s'appuyant sur un log de fichiers déjà traités.
- Ce pipeline permet d’ajouter progressivement des spectres au jeu d’entraînement, en assurant qu’aucun spectre ne soit traité deux fois. Chaque exécution sélectionne un nouveau lot, l’entraîne, puis marque les spectres comme utilisés.


#

## Étape 0 : SETUP & IMPORTS

In [ ]:
from utils import setup_project_env
from pipeline.master import MasterPipeline

# Initialisation automatique de l'environnement et des chemins
paths = setup_project_env()

# Instanciation du pipeline maître
pipeline = MasterPipeline(
    raw_data_dir=paths["RAW_DATA_DIR"],
    catalog_dir=paths["CATALOG_DIR"],
    processed_dir=paths["PROCESSED_DIR"],
    models_dir=paths["MODELS_DIR"],
    reports_dir=paths["REPORTS_DIR"],
)

print("\nSetup terminé. Tu es prêt à lancer ton pipeline.")

#

## Faire fonctionner la pipeline d'un début à la fin

Lancer la cellule ci-bas pour faire fonctionner la pipeline au complet d'une seule ligne de commande, sinon suivre plus bas pour faire étape par étape!

### - Entraîner un RandomForest
``pipeline.run_full_pipeline(batch_size=2000, model_type='RandomForest')``

### - Entraîner un XGBoost
``pipeline.run_full_pipeline(batch_size=2000, model_type='XGBoost')``

In [ ]:
pipeline.run_full_pipeline(batch_size=500, model_type='XGBoost')

#

## 1) Téléchargement des spectres
Utilisation du script ``dr5_downloader.py`` encapsulé en fonction.

Cette étape est désormais externalisée dans [01_download_spectra.ipynb](./01_download_spectra.ipynb) pour être exécutée seulement au besoin.

#

## 2) Préparation des données spectrales : Sélection du lot de spectres à traiter
- Le DatasetBuilder sélectionne un lot de nouveaux spectres jamais utilisés.
- Si tous les spectres disponibles ont déjà été utilisés, le pipeline s'arrête proprement.


In [ ]:
# Pour définir la grosseur du lot modifier la variable de batch_size= par la valeur voulu
pipeline._select_batch(batch_size=5000)

#

## 3) parcours ce lot de spectres pour générer un CSV à partir des headers des fichiers ``.fits.gz`` et Exploration des features
C’est ce CSV qui sera ton ``master_catalog`` local, aligné exactement avec les spectres que tu vas traiter dans ce lot.

### Analyse exploratoire des spectres prétraités
Visualiser SNR, distribution de classes, etc.

In [ ]:
pipeline._generate_local_catalog()
pipeline._process_batch()

#

## 4) Entraînement du modèle
### Machine Learning
Sélection des features et entraînement d'un modèle de classification.

In [ ]:
pipeline.interactive_training_runner()